In [ ]:
!time pip install rdkit-pypi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 36.1 MB/s eta 0:00:00

real	0m6.076s
user	0m4.250s
sys	0m0.471s


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# import seaborn as sn
# import scipy as sp

import random
import tensorflow
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, BatchNormalization, Conv2D, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

from rdkit import Chem # A core definition
from rdkit.Chem.Draw import MolsToGridImage # For displaying multiple molecules
from rdkit.Chem.AllChem import * # conformer generation and adding H
from rdkit.Chem.rdMolDescriptors import * # To calculate descriptors
from rdkit.Chem.Draw import IPythonConsole # This displays the molecule in-line
from rdkit.Chem import PandasTools # for pandas dataframe with rdkit
from rdkit.DataStructs.cDataStructs import ConvertToNumpyArray
import pandas.io.formats.style

# Dual active models

In [ ]:
############################################Sexual data

import pandas as pd
sxl= pd.read_csv("M_sexual_LibrariesC_hits_nd_2ndclst_nhits.csv")
sxl['S_Hit'] = sxl['S_Hit'].replace(np.nan, "No")
sxl = sxl.iloc[: , 1:]
print(sxl['S_Hit'].value_counts())
sxl

## MACCSKeysFingerprint

In [ ]:
!pip install deepchem

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 693.2/693.2 KB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 MB 19.7 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.10.1
    Uninstalling scipy-1.10.1:
      Successfully uninstalled scipy-1.10.1


In [ ]:
import deepchem as dc
from deepchem.feat import MACCSKeysFingerprint

In [ ]:
sxl["Target"] = np.where(sxl["S_Hit"]== "Yes", 1,0)
sxl["Target"].value_counts()

0    8975
1     916
Name: Target, dtype: int64

In [ ]:
dchm= sxl["Smiles"]
mols = [Chem.MolFromSmiles(x) for x in dchm]
featurizer = MACCSKeysFingerprint()
maccs = featurizer.featurize(mols)
df=[]
df = pd.DataFrame(maccs)
df

0       NC(=O)c1cccc2nc([C@H]3CCN(Cc4ccccn4)C3)[nH]c12
1                Fc1ccc2[nH]c(C3CCCN3Cc3c[nH]cn3)nc2c1
2                        COc1c(C)cnc(CN(C)C2CCCCC2)c1C
3                Cc1cccc(CN2CCCC2c2cccc(Nc3nccs3)n2)c1
4                   CCSCCNc1nc(-c2ccccn2)nc2c1c(C)nn2C
                             ...                      
9886          Cc1nc(C)n(C2CCCN(C(=O)c3ncoc3C(C)C)C2)n1
9887              COCCN1CCN(Cc2c(C)noc2C)Cc2ccc(C)nc21
9888               CCc1ncnc2c1CCN(Cc1ccc3c(c1)OCO3)CC2
9889              CCC1CN(CC(=O)NCCN2CCCCCC2)c2ccccc2S1
9890                Cc1cnn(CC2CCCN2Cc2nc3ccccc3nc2C)c1
Name: Smiles, Length: 9891, dtype: object

In [ ]:
from sklearn.model_selection import train_test_split
Xt_train, Xt_test, Yt_train, Yt_test = train_test_split(df, sxl["Target"],test_size=0.2, random_state=42)

# Hyperparameter Tuning
Logistic regression does not really have any critical hyperparameters to tune.

Sometimes, you can see useful differences in performance or convergence with different solvers (solver).

Regularization (penalty) can sometimes be helpful.


Note: not all solvers support all regularization terms.

The C parameter controls the penality strength, which can also be effective.


---

Logistic Regression Hyperparameters we’ll be Looking at:

*   solver in [‘newton-cg’, ‘lbfgs’, ‘liblinear’, ‘sag’, ‘saga’]
*   penalty in [‘none’, ‘l1’, ‘l2’, ‘elasticnet’]
*   C in [100, 10, 1.0, 0.1, 0.01]
---
https://machinelearningmastery.com/hyperparameters-for-classification-machine-learning-algorithms/
https://www.projectpro.io/recipes/optimize-hyper-parameters-of-logistic-regression-model-using-grid-search-in-python

In [ ]:
# example of grid searching key hyperparametres for logistic regression
from sklearn.datasets import make_blobs
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
# define dataset
#X = mfpdf  # we only take the first three features. iF THESE FEATURES ARE NOT NB CHANGE!!
#y = sxl["Target"]

# define models and parameters
model = LogisticRegression(class_weight={0:1,1:10})
solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['l2']
c_values = [100, 10, 1.0, 0.1, 0.01]
# define grid search
grid = dict(solver=solvers,penalty=penalty,C=c_values)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(Xt_train, Yt_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.932802 using {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
0.932507 (0.007931) with: {'C': 100, 'penalty': 'l2', 'solver': 'newton-cg'}
0.932802 (0.007931) with: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
0.932550 (0.007957) with: {'C': 100, 'penalty': 'l2', 'solver': 'liblinear'}
0.932465 (0.007588) with: {'C': 10, 'penalty': 'l2', 'solver': 'newton-cg'}
0.932634 (0.007458) with: {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}
0.932423 (0.007540) with: {'C': 10, 'penalty': 'l2', 'solver': 'liblinear'}
0.931791 (0.006746) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'newton-cg'}
0.931412 (0.006749) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'lbfgs'}
0.931623 (0.006729) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'liblinear'}
0.926821 (0.005996) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'newton-cg'}
0.926947 (0.005942) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
0.927031 (0.005857) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}
0.915066 (0.005554) with: {

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [ ]:
print(grid_result.best_params_)
print(grid_result.best_score_)

{'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
0.9328019448594669


In [ ]:
print(grid_result.cv_results_)

{'mean_fit_time': array([8.40517854, 0.45190898, 1.16819469, 3.10810455, 0.47743727,
       0.67710054, 2.68917503, 0.81298531, 0.68315075, 1.32306056,
       0.57417886, 0.3973692 , 0.58186402, 0.20752591, 0.19879614]), 'std_fit_time': array([1.49309663, 0.06990668, 0.10635643, 0.39575093, 0.12745302,
       0.06103971, 0.8291319 , 0.22368519, 0.13231302, 0.30059905,
       0.08222324, 0.07412562, 0.09257946, 0.0235857 , 0.07239124]), 'mean_score_time': array([0.00676936, 0.00683155, 0.00625068, 0.00676606, 0.00679881,
       0.00656432, 0.01291064, 0.00986228, 0.01128521, 0.01047777,
       0.00878241, 0.0124241 , 0.00807822, 0.00672303, 0.01009882]), 'std_score_time': array([0.00109654, 0.00159397, 0.00048522, 0.00054305, 0.00143234,
       0.00201218, 0.00776329, 0.0037869 , 0.00542367, 0.00470296,
       0.00449897, 0.00592741, 0.00424868, 0.00107207, 0.00532329]), 'param_C': masked_array(data=[100, 100, 100, 10, 10, 10, 1.0, 1.0, 1.0, 0.1, 0.1,
                   0.1, 0.01, 0.01,

In [ ]:
clf_predict = grid_result.predict(Xt_test)

In [ ]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.932802 using {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
0.932507 (0.007931) with: {'C': 100, 'penalty': 'l2', 'solver': 'newton-cg'}
0.932802 (0.007931) with: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
0.932550 (0.007957) with: {'C': 100, 'penalty': 'l2', 'solver': 'liblinear'}
0.932465 (0.007588) with: {'C': 10, 'penalty': 'l2', 'solver': 'newton-cg'}
0.932634 (0.007458) with: {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}
0.932423 (0.007540) with: {'C': 10, 'penalty': 'l2', 'solver': 'liblinear'}
0.931791 (0.006746) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'newton-cg'}
0.931412 (0.006749) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'lbfgs'}
0.931623 (0.006729) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'liblinear'}
0.926821 (0.005996) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'newton-cg'}
0.926947 (0.005942) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
0.927031 (0.005857) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}
0.915066 (0.005554) with: {

# Training

In [ ]:
from sklearn.linear_model import  LogisticRegression

lrc = LogisticRegression(class_weight={0:1,1:10}, max_iter=300, C= 100, penalty= 'l2', solver= 'lbfgs')

# fit the predictor and target
fit=lrc.fit(Xt_train, Yt_train)

# predict
lr_predict = lrc.predict(Xt_test)# check performance

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
# Save Model Using joblib
import pandas
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
import joblib

%cd /content/
# save the model to disk
filename = 'sexualsLR_HPTFInal_UMACCS_model.pkl'
joblib.dump(lrc, filename)


/content
0.9196563921172309


#Load sexual screening test set

In [ ]:
import pandas as pd
df1= pd.read_csv("sexualsUntrainedTest.csv")
df1

,Unnamed: 0,mol,S_Hit,Smiles
0,25658,<rdkit.Chem.rdchem.Mol object at 0x7fc9363b4c10>,No,Cn1cc(CN2CC[C@@H]3[C@@H](CCC(=O)N3CCc3c[nH]cn3...
1,39559,<rdkit.Chem.rdchem.Mol object at 0x7fc9362189e0>,No,CC1(C)CN=C(N2CCN(CC3CCOC3)CC2)S1
2,36544,<rdkit.Chem.rdchem.Mol object at 0x7fc936195770>,No,COCCC(=O)N1CCN(CC2CC2)c2ccccc21
3,58789,<rdkit.Chem.rdchem.Mol object at 0x7fc935d5bd80>,No,CC(CC(=O)O)CC(=O)Nc1sc2c(c1C(N)=O)CCc1ccccc1-2
4,15213,<rdkit.Chem.rdchem.Mol object at 0x7fc9365e8190>,No,Clc1ccccc1CN1CCc2nc(C3CCCCN3)ncc2C1
...,...,...,...,...
62370,50615,<rdkit.Chem.rdchem.Mol object at 0x7fc935ffa1f0>,No,CC[C@@H](c1ccccc1)n1c(C2CC2)n[nH]c1=O
62371,57063,<rdkit.Chem.rdchem.Mol object at 0x7fc935f145f0>,No,CCn1ncc(S(=O)(=O)N[C@@H]2CC[C@@H](n3ccnc3C)[C@...
62372,19513,<rdkit.Chem.rdchem.Mol object at 0x7fc9364a6ff0>,No,O=C(Nc1ccc(F)cc1)C1CNCCN1C(=O)CC1CCCCC1
62373,41985,<rdkit.Chem.rdchem.Mol object at 0x7fc93607fa00>,No,CNC(=O)CN1CCC(NC(=O)CC(C)CCc2ccccc2)CC1


In [ ]:
compounds = []


for _, compound_key, smiles in df1[["S_Hit", "Smiles"]].itertuples():
    compounds.append((Chem.MolFromSmiles(smiles), compound_key, smiles))


In [ ]:
dff= pd.DataFrame (compounds, columns = ['mol', 'S_Hit', "Smiles"])
sxl=dff
sxl

,mol,S_Hit,Smiles
0,<rdkit.Chem.rdchem.Mol object at 0x7f71c322c9e0>,No,Cn1cc(CN2CC[C@@H]3[C@@H](CCC(=O)N3CCc3c[nH]cn3...
1,<rdkit.Chem.rdchem.Mol object at 0x7f71c322d000>,No,CC1(C)CN=C(N2CCN(CC3CCOC3)CC2)S1
2,<rdkit.Chem.rdchem.Mol object at 0x7f71c322c580>,No,COCCC(=O)N1CCN(CC2CC2)c2ccccc21
3,<rdkit.Chem.rdchem.Mol object at 0x7f71c322c740>,No,CC(CC(=O)O)CC(=O)Nc1sc2c(c1C(N)=O)CCc1ccccc1-2
4,<rdkit.Chem.rdchem.Mol object at 0x7f71c322c7b0>,No,Clc1ccccc1CN1CCc2nc(C3CCCCN3)ncc2C1
...,...,...,...
62370,<rdkit.Chem.rdchem.Mol object at 0x7f71c43d49e0>,No,CC[C@@H](c1ccccc1)n1c(C2CC2)n[nH]c1=O
62371,<rdkit.Chem.rdchem.Mol object at 0x7f71c43d4a50>,No,CCn1ncc(S(=O)(=O)N[C@@H]2CC[C@@H](n3ccnc3C)[C@...
62372,<rdkit.Chem.rdchem.Mol object at 0x7f71c43d4ac0>,No,O=C(Nc1ccc(F)cc1)C1CNCCN1C(=O)CC1CCCCC1
62373,<rdkit.Chem.rdchem.Mol object at 0x7f71c43d4b30>,No,CNC(=O)CN1CCC(NC(=O)CC(C)CCc2ccccc2)CC1


In [ ]:
!pip install deepchem

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:

dpchm= sxl
dpchm
dpchm["Target"] = np.where(dpchm["S_Hit"]== "Yes", 1,0)
dpchm["Target"].value_counts()

dchm= dpchm["mol"]
import deepchem as dc
from deepchem.feat import MACCSKeysFingerprint
#mols = [Chem.MolFromSmiles(x) for x in dchm]
featurizer = MACCSKeysFingerprint()
maccs = featurizer.featurize(dchm)
df=[]
df = pd.DataFrame(maccs)

df

,0,1,2,3,4,5,6,7,8,9,...,157,158,159,160,161,162,163,164,165,166
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,1,1,1,0,1,0
1,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,1,1,1,1,1,0
2,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,1,1,1,1,1,0
4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,1,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62370,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,1,1,0,1,1,0
62371,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,1,1,1,1,1,0
62372,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
62373,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0


In [ ]:
Xt_test = df
Yt_test= sxl["Target"]

#LR

In [ ]:

lr_model = joblib.load('sexualsLR_HPTFInal_UMACCS_model.pkl')

In [ ]:

lrc_predict = lr_model.predict(Xt_test)
p= lrc_predict
p= pd.DataFrame(p)
print(p.value_counts())

0    57975
1     4400
dtype: int64


In [ ]:
from sklearn.metrics import recall_score, precision_recall_curve,precision_score, confusion_matrix, accuracy_score, f1_score
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(Yt_test,lrc_predict)
print('Accuracy: %f' % accuracy)
recall= recall_score(Yt_test,lrc_predict)
print("Recall %f" % recall)
precision= precision_score(Yt_test,lrc_predict)
print("precision %f" % precision)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(Yt_test,lrc_predict)
print('F1 score: %f' % f1)
cm= confusion_matrix(Yt_test,lrc_predict)
cm

Accuracy: 0.930501
Recall 0.777778
precision 0.020682
F1 score: 0.040292


array([[57949,  4309],
       [   26,    91]])

In [ ]:
from sklearn.metrics import recall_score, precision_recall_curve,precision_score, confusion_matrix, accuracy_score, f1_score, roc_auc_score
print('ROCAUC score:',roc_auc_score(Yt_test, lrc_predict))
print('Accuracy score:',accuracy_score(Yt_test, lrc_predict))
print('F1 score:',f1_score(Yt_test, lrc_predict))

ROCAUC score: 0.854282894478532
Accuracy score: 0.930501002004008
F1 score: 0.04029222935576711


In [ ]:
from sklearn.metrics import recall_score, precision_recall_curve,precision_score, confusion_matrix, accuracy_score, f1_score, roc_auc_score, cohen_kappa_score
# kappa
kappa = cohen_kappa_score(Yt_test, lrc_predict)
print('Cohens kappa: %f' % kappa)
# ROC AUC
auc = roc_auc_score(Yt_test, lrc_predict)
print('ROC AUC: %f' % auc)
# confusion matrix
matrix = confusion_matrix(Yt_test, lrc_predict)
print(matrix)

Cohens kappa: 0.036772
ROC AUC: 0.854283
[[57949  4309]
 [   26    91]]


In [ ]:
print(f"Test Set: {len(Xt_test)}")
print(Yt_test.value_counts())
print(matrix)

Test Set: 62375
0    62258
1      117
Name: Target, dtype: int64
[[57949  4309]
 [   26    91]]


# Asexual Models

In [ ]:
import pandas as pd
axl= pd.read_csv("M_asexuals_lib_hits_nd_clst_nhts.csv")
axl['Hit'] = axl['Hit'].replace(np.nan, "No")
axl = axl.iloc[: , 1:]
print(axl['Hit'].value_counts())
axl

In [ ]:
compounds =[]
df1= axl[:20000]

for _, smiles, hit in df1[["Smiles", "Hit"]].itertuples():
    compounds.append((Chem.MolFromSmiles(smiles),hit))

df2= axl[20001:23766]

for _, smiles, hit in df2[["Smiles", "Hit"]].itertuples():
    compounds.append((Chem.MolFromSmiles(smiles),hit))

df3= axl[23767:25396]

for _, smiles, hit in df3[["Smiles", "Hit"]].itertuples():
    compounds.append((Chem.MolFromSmiles(smiles),hit))


df4= axl[25397:25662]

for _, smiles, hit in df4[["Smiles", "Hit"]].itertuples():
    compounds.append((Chem.MolFromSmiles(smiles),hit))

df5= axl[25663:26912]

for _, smiles, hit in df5[["Smiles", "Hit"]].itertuples():
    compounds.append((Chem.MolFromSmiles(smiles),hit))

df6= axl[26913:28399]

for _, smiles, hit in df6[["Smiles", "Hit"]].itertuples():
    compounds.append((Chem.MolFromSmiles(smiles),hit))

df7= axl[28400:]

for _, smiles, hit in df7[["Smiles", "Hit"]].itertuples():
    compounds.append((Chem.MolFromSmiles(smiles),hit))

len(compounds)

59530

In [ ]:
axl["Target"] = np.where(axl["Hit"]== "Yes", 1,0)
axl["Target"].value_counts()

1    30393
0    29143
Name: Target, dtype: int64

In [ ]:
compounds
dpchm= pd.DataFrame (compounds, columns = ['mol', 'hit'])
dpchm

,mol,hit
0,<rdkit.Chem.rdchem.Mol object at 0x7f85fd6f5820>,Yes
1,<rdkit.Chem.rdchem.Mol object at 0x7f85fd6f5dd0>,Yes
2,<rdkit.Chem.rdchem.Mol object at 0x7f85fd6f5e40>,Yes
3,<rdkit.Chem.rdchem.Mol object at 0x7f85fd6f5eb0>,Yes
4,<rdkit.Chem.rdchem.Mol object at 0x7f85fd6f5f20>,Yes
...,...,...
59525,<rdkit.Chem.rdchem.Mol object at 0x7f85fa201580>,No
59526,<rdkit.Chem.rdchem.Mol object at 0x7f85fa2015f0>,No
59527,<rdkit.Chem.rdchem.Mol object at 0x7f85fa201660>,No
59528,<rdkit.Chem.rdchem.Mol object at 0x7f85fa2016d0>,No


## MACCSKeysFingerprint

In [ ]:
!pip install deepchem

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#dchm= axl["Smiles"]
#dchm
dpchm["Target"] = np.where(dpchm["hit"]== "Yes", 1,0)
dpchm["Target"].value_counts()

dchm= dpchm["mol"]
dchm

0        <rdkit.Chem.rdchem.Mol object at 0x7f85fd6f5820>
1        <rdkit.Chem.rdchem.Mol object at 0x7f85fd6f5dd0>
2        <rdkit.Chem.rdchem.Mol object at 0x7f85fd6f5e40>
3        <rdkit.Chem.rdchem.Mol object at 0x7f85fd6f5eb0>
4        <rdkit.Chem.rdchem.Mol object at 0x7f85fd6f5f20>
                               ...                       
59525    <rdkit.Chem.rdchem.Mol object at 0x7f85fa201580>
59526    <rdkit.Chem.rdchem.Mol object at 0x7f85fa2015f0>
59527    <rdkit.Chem.rdchem.Mol object at 0x7f85fa201660>
59528    <rdkit.Chem.rdchem.Mol object at 0x7f85fa2016d0>
59529    <rdkit.Chem.rdchem.Mol object at 0x7f85fa201740>
Name: mol, Length: 59530, dtype: object

In [ ]:
import deepchem as dc
from deepchem.feat import MACCSKeysFingerprint
#mols = [Chem.MolFromSmiles(x) for x in dchm]
featurizer = MACCSKeysFingerprint()
maccs = featurizer.featurize(dchm)
df=[]
df = pd.DataFrame(maccs)
#df["Target"]= sxl["Target"]
df

array([[0, 0, 0, ..., 1, 1, 0],
       [0, 0, 0, ..., 1, 1, 0],
       [0, 0, 0, ..., 1, 1, 0],
       ...,
       [0, 0, 0, ..., 1, 1, 0],
       [0, 0, 0, ..., 1, 1, 0],
       [0, 0, 0, ..., 1, 1, 0]])

In [ ]:
from sklearn.model_selection import train_test_split
Xt_train, Xt_test, Yt_train, Yt_test = train_test_split(df, dpchm['hit'],test_size=0.2, random_state=42)

In [ ]:
Yt_train= np.where(Yt_train== "Yes", 1,0)
Yt_test= np.where(Yt_test== "Yes", 1,0)

# Hyperparameter Tuning
Logistic regression does not really have any critical hyperparameters to tune.

Sometimes, you can see useful differences in performance or convergence with different solvers (solver).

Regularization (penalty) can sometimes be helpful.


Note: not all solvers support all regularization terms.

The C parameter controls the penality strength, which can also be effective.


---

Logistic Regression Hyperparameters we’ll be Looking at:

*   solver in [‘newton-cg’, ‘lbfgs’, ‘liblinear’, ‘sag’, ‘saga’]
*   penalty in [‘none’, ‘l1’, ‘l2’, ‘elasticnet’]
*   C in [100, 10, 1.0, 0.1, 0.01]
---
https://machinelearningmastery.com/hyperparameters-for-classification-machine-learning-algorithms/
https://www.projectpro.io/recipes/optimize-hyper-parameters-of-logistic-regression-model-using-grid-search-in-python

In [ ]:
# example of grid searching key hyperparametres for logistic regression
from sklearn.datasets import make_blobs
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
# define dataset
#X = mfpdf  # we only take the first three features. iF THESE FEATURES ARE NOT NB CHANGE!!
#y = sxl["Target"]

# define models and parameters
model = LogisticRegression(class_weight={0:1,1:10})
solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['l2']
c_values = [100, 10, 1.0, 0.1, 0.01]
# define grid search
grid = dict(solver=solvers,penalty=penalty,C=c_values)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(Xt_train, Yt_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.796881 using {'C': 100, 'penalty': 'l2', 'solver': 'newton-cg'}
0.796881 (0.004611) with: {'C': 100, 'penalty': 'l2', 'solver': 'newton-cg'}
0.796797 (0.004414) with: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
0.796846 (0.004607) with: {'C': 100, 'penalty': 'l2', 'solver': 'liblinear'}
0.796804 (0.004631) with: {'C': 10, 'penalty': 'l2', 'solver': 'newton-cg'}
0.796454 (0.004492) with: {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}
0.796776 (0.004551) with: {'C': 10, 'penalty': 'l2', 'solver': 'liblinear'}
0.796405 (0.004695) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'newton-cg'}
0.796202 (0.004581) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'lbfgs'}
0.796342 (0.004636) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'liblinear'}
0.793494 (0.004805) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'newton-cg'}
0.793592 (0.004889) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
0.793424 (0.004818) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}
0.780909 (0.004850) wit

In [ ]:
print(grid_result.best_params_)
print(grid_result.best_score_)

{'C': 100, 'penalty': 'l2', 'solver': 'newton-cg'}
0.7968811560153429


In [ ]:
print(grid_result.cv_results_)

{'mean_fit_time': array([104.5611939 ,   3.4813815 ,   5.97326674,  42.05335616,
         3.27006437,   5.77301851,  20.93503158,   3.33779103,
         4.03321033,  12.18770592,   3.43181316,   2.23505938,
         9.19611352,   3.38861965,   1.59495497]), 'std_fit_time': array([13.88152628,  0.141883  ,  0.47801516,  2.91188726,  0.10465561,
        0.68381362,  1.37930945,  0.08706555,  0.21806052,  0.55472994,
        0.08159789,  0.09771313,  0.49491653,  0.09284972,  0.06041158]), 'mean_score_time': array([0.01002027, 0.00980151, 0.00855091, 0.00950872, 0.0090286 ,
       0.00816501, 0.00961796, 0.00919365, 0.00876122, 0.00929492,
       0.00928526, 0.00882415, 0.01044439, 0.00913021, 0.00832291]), 'std_score_time': array([0.00035535, 0.00267792, 0.00054726, 0.00087429, 0.00195709,
       0.00017826, 0.00149733, 0.0024789 , 0.00146167, 0.00071396,
       0.00127277, 0.00071318, 0.0018725 , 0.00149792, 0.00092437]), 'param_C': masked_array(data=[100, 100, 100, 10, 10, 10, 1.0, 1.0

In [ ]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.796881 using {'C': 100, 'penalty': 'l2', 'solver': 'newton-cg'}
0.796881 (0.004611) with: {'C': 100, 'penalty': 'l2', 'solver': 'newton-cg'}
0.796797 (0.004414) with: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
0.796846 (0.004607) with: {'C': 100, 'penalty': 'l2', 'solver': 'liblinear'}
0.796804 (0.004631) with: {'C': 10, 'penalty': 'l2', 'solver': 'newton-cg'}
0.796454 (0.004492) with: {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}
0.796776 (0.004551) with: {'C': 10, 'penalty': 'l2', 'solver': 'liblinear'}
0.796405 (0.004695) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'newton-cg'}
0.796202 (0.004581) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'lbfgs'}
0.796342 (0.004636) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'liblinear'}
0.793494 (0.004805) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'newton-cg'}
0.793592 (0.004889) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
0.793424 (0.004818) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}
0.780909 (0.004850) wit

# Training

In [ ]:
###################################################################### Logistic Regression adjusting classweights (https://www.kdnuggets.com/2020/01/5-most-useful-techniques-handle-imbalanced-datasets.html)
from sklearn.linear_model import  LogisticRegression
###Best: 0.796881 using {'C': 100, 'penalty': 'l2', 'solver': 'newton-cg'}
lrc = LogisticRegression(class_weight={0:1,1:10}, C= 100, penalty= "l2", solver= "newton-cg")

# fit the predictor and target
lrc.fit(Xt_train, Yt_train)

# predict
lr_predict = lrc.predict(Xt_test)# check performance

In [ ]:
# Save Model Using joblib
import pandas
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
import joblib

%cd /content/
# save the model to disk
filename = 'asexualsLR_HPTFInal_UMACCS_model.pkl'
joblib.dump(lrc, filename)

#Load Asexual screening test set

In [ ]:
import pandas as pd
df1= pd.read_csv("asexualsUntrainedTest.csv")
df1

,Unnamed: 0,mol,Hit,Smiles
0,51742,<rdkit.Chem.rdchem.Mol object at 0x7f71149bf680>,No,O=C(NCCCN1CCOCC1)C1CCCN(Cc2ccco2)C1
1,20279,<rdkit.Chem.rdchem.Mol object at 0x7f7114ed6180>,No,Cc1nocc1C(=O)N1CCCC2(CCN(c3cnn(C)c3)C2=O)C1
2,17729,<rdkit.Chem.rdchem.Mol object at 0x7f71219e38b0>,No,Cn1cncc1-c1nnc([C@H]2CCN(Cc3cccnc3)C2)o1
3,28545,<rdkit.Chem.rdchem.Mol object at 0x7f7114dc26c0>,No,CCn1nc(C)c2nc(C(C)C)nc(NCC(C)C)c21
4,24371,<rdkit.Chem.rdchem.Mol object at 0x7f7114f4b140>,No,CN(C)CC1(O)CCCN(Cc2ccccc2OCC(=O)N(C)C)C1
...,...,...,...,...
61024,72080,<rdkit.Chem.rdchem.Mol object at 0x7f7114740cf0>,No,N#Cc1cccc(NC(=O)NC2CCN(CCCCCNC(=O)/C=C/c3ccc(C...
61025,49244,<rdkit.Chem.rdchem.Mol object at 0x7f7114b4a340>,No,NC1(C(=O)NCC2CCCN(Cc3cccc4nonc34)C2)CCC1
61026,47413,<rdkit.Chem.rdchem.Mol object at 0x7f7114afbae0>,No,CNC(=O)C1CCCN(c2nc(-c3ccco3)nc3c2CCNCC3)C1
61027,73164,<rdkit.Chem.rdchem.Mol object at 0x7f711476eb20>,No,CC(Cc1ccc(OC2CCCCC2)cc1)NCC(O)c1cccc(Cl)c1.Cl


In [ ]:
compounds = []


for _, compound_key, smiles in df1[["Hit", "Smiles"]].itertuples():
    compounds.append((Chem.MolFromSmiles(smiles), compound_key, smiles))


In [ ]:
dff= pd.DataFrame (compounds, columns = ['mol', 'Hit', "Smiles"])
axl=dff
axl

,mol,S_Hit,Smiles
0,<rdkit.Chem.rdchem.Mol object at 0x7f7113906500>,No,O=C(NCCCN1CCOCC1)C1CCCN(Cc2ccco2)C1
1,<rdkit.Chem.rdchem.Mol object at 0x7f7113906960>,No,Cc1nocc1C(=O)N1CCCC2(CCN(c3cnn(C)c3)C2=O)C1
2,<rdkit.Chem.rdchem.Mol object at 0x7f7113907680>,No,Cn1cncc1-c1nnc([C@H]2CCN(Cc3cccnc3)C2)o1
3,<rdkit.Chem.rdchem.Mol object at 0x7f71139061f0>,No,CCn1nc(C)c2nc(C(C)C)nc(NCC(C)C)c21
4,<rdkit.Chem.rdchem.Mol object at 0x7f7113906c00>,No,CN(C)CC1(O)CCCN(Cc2ccccc2OCC(=O)N(C)C)C1
...,...,...,...
61024,<rdkit.Chem.rdchem.Mol object at 0x7f7112d2ef80>,No,N#Cc1cccc(NC(=O)NC2CCN(CCCCCNC(=O)/C=C/c3ccc(C...
61025,<rdkit.Chem.rdchem.Mol object at 0x7f7112d2eff0>,No,NC1(C(=O)NCC2CCCN(Cc3cccc4nonc34)C2)CCC1
61026,<rdkit.Chem.rdchem.Mol object at 0x7f7112d2f060>,No,CNC(=O)C1CCCN(c2nc(-c3ccco3)nc3c2CCNCC3)C1
61027,<rdkit.Chem.rdchem.Mol object at 0x7f7112d2f0d0>,No,CC(Cc1ccc(OC2CCCCC2)cc1)NCC(O)c1cccc(Cl)c1.Cl


## MACCSKeysFingerprint

In [ ]:
!pip install deepchem

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
dpchm= axl
#dchm
dpchm["Target"] = np.where(dpchm["Hit"]== "Yes", 1,0)
dpchm["Target"].value_counts()

dchm= dpchm["mol"]
import deepchem as dc
from deepchem.feat import MACCSKeysFingerprint
#mols = [Chem.MolFromSmiles(x) for x in dchm]
featurizer = MACCSKeysFingerprint()
maccs = featurizer.featurize(dchm)
df=[]
df = pd.DataFrame(maccs)
#df["Target"]= sxl["Target"]

df

,0,1,2,3,4,5,6,7,8,9,...,157,158,159,160,161,162,163,164,165,166
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,1,1,1,1,0
1,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,1,1,1,1,1,0
2,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,1,1,1,1,1,0
4,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,1,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61024,0,0,0,0,0,0,0,0,0,0,...,1,1,1,0,1,1,1,1,1,0
61025,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
61026,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
61027,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,1,1,1,1,1,0


In [ ]:
Xt_test = df
Yt_test= axl["Target"]

#LR

In [ ]:
import pandas
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
import joblib

In [ ]:

lr_model = joblib.load('asexualsLR_HPTFInal_UMACCS_model.pkl')

In [ ]:
lrc_predict = lr_model.predict(Xt_test)
p= lrc_predict
p= pd.DataFrame(p)
print(p.value_counts())

0    31588
1    29441
dtype: int64


In [ ]:
from sklearn.metrics import recall_score, precision_recall_curve,precision_score, confusion_matrix, accuracy_score, f1_score
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(Yt_test,lrc_predict)
print('Accuracy: %f' % accuracy)
recall= recall_score(Yt_test,lrc_predict)
print("Recall %f" % recall)
precision= precision_score(Yt_test,lrc_predict)
print("precision %f" % precision)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(Yt_test,lrc_predict)
print('F1 score: %f' % f1)
cm= confusion_matrix(Yt_test,lrc_predict)
cm

Accuracy: 0.560750
Recall 0.972382
precision 0.092082
F1 score: 0.168234


array([[31511, 26730],
       [   77,  2711]])

In [ ]:
from sklearn.metrics import recall_score, precision_recall_curve,precision_score, confusion_matrix, accuracy_score, f1_score, roc_auc_score
print('ROCAUC score:',roc_auc_score(Yt_test, lrc_predict))
print('Accuracy score:',accuracy_score(Yt_test, lrc_predict))
print('F1 score:',f1_score(Yt_test, lrc_predict))

ROCAUC score: 0.7567133019511737
Accuracy score: 0.5607498074685805
F1 score: 0.16823357845418724


In [ ]:
from sklearn.metrics import recall_score, precision_recall_curve,precision_score, confusion_matrix, accuracy_score, f1_score, roc_auc_score, cohen_kappa_score
# kappa
kappa = cohen_kappa_score(Yt_test, lrc_predict)
print('Cohens kappa: %f' % kappa)
# ROC AUC
auc = roc_auc_score(Yt_test, lrc_predict)
print('ROC AUC: %f' % auc)
# confusion matrix
matrix = confusion_matrix(Yt_test, lrc_predict)
print(matrix)

Cohens kappa: 0.092490
ROC AUC: 0.756713
[[31511 26730]
 [   77  2711]]


In [ ]:
print(f"Test Set: {len(Xt_test)}")
print(Yt_test.value_counts())
print(matrix)

Test Set: 61029
0    58241
1     2788
Name: Target, dtype: int64
[[31511 26730]
 [   77  2711]]
